### modules

In [ ]:
if globals().get('LOADED_LOGGER') == None:
    %run LOGGER.ipynb

In [ ]:
if globals().get('LOADED_ANNOTATIONS') == None:
    %run ANNOTATIONS.ipynb

In [ ]:
if globals().get('LOADED_PATTERN_OBSERVER') == None:
    %run PATTERN_OBSERVER.ipynb

In [ ]:
if globals().get('VIEWING_TABS') == None:
    %run VIEWING_TABS.ipynb # dropdown needs to know what plotting options are

### import protection

In [ ]:
if globals().get('LOADED_SETTINGS') == None:
    logging.info('LOADED_SETTINGS')
    LOADED_SETTINGS=True

### imports

In [ ]:
from ipywidgets import VBox,HBox,Text,DatePicker,Button,Layout,Dropdown,Output
from ipywidgets import Color # pretty color for apply button
from pathlib import Path # for parsing directory contents
from IPython.display import Javascript
import os, signal # kill process
#from IPython.display import display,Javascript

In [ ]:
from datetime import time
from ipydatetime import TimePicker
'''
https://ipydatetime.readthedocs.io/en/latest/installing.html
conda install -c conda-forge nodejs
conda install ipydatetime
jupyter labextension install jupyter-widget-datetime
jupyter labextension install jupyter-widget-datetime @jupyter-widgets/jupyterlab-manager
'''
None

### begin

In [ ]:
'''
SUBJECT
- CHECKBOXES observes SETTINGS for options
- DATA_CONTAINER observes SETTINGS for repopulation purposes
'''
class SETTINGS(ISubject):

    _default_app_name      = 'Backtest Viewer'
    
    _default_backtest_path = r'C:\Users\ahkar\OneDrive\Documents\Data\Dev\B3'
    
    _view_types = [
        'Extraday',
        'Intraday',
        'Factor',
    ]
    
    _return_types = [
        'Cash + Future',
        'Cash',
        'Future',
    ]
    
    '''SETTINGS._option_names need to be instantiable widgets, dico value denotes if gets reset'''
    _option_names = {
        'app_name'      : False,
        'backtest_path' : False,
        'view_type'     : False,
        'plot_method'   : True,
        'return_type'   : True,
        'date_from'     : True,
        'date_to'       : True,
        'time_from'     : True,
        'time_to'       : True,
    }

    '''
    min time + max time
    https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.indexer_between_time.html
    '''
    ####################################
    # constructor
    ####################################
    def __init__(self,
        name          = 'SETTINGS',
        width         = '25%',
        ):
                
        self._width                         = width
        
        self.name                           = name
        self.widget                         = self._make_widget()
        
        # for observer pattern
        self.reference                      = {}                             # dico holds all useful info
        self._deltas                        = {}                             # dico holds change
        self._observers                     = []
        
        # update checkpoint
        self._default_settings              = self._current_settings
        self.applied_settings               = self._current_settings
        self._update_reference(True)                                         # force populate all on first run

    ####################################
    # observer pattern
    ####################################
    # subject
    def attach(self,observer : IObserver) -> None :
        logging.info(f'OBSERVER PATTERN : {observer.name} : OBSERVES {self.name}')
        self._observers.append(observer)
        
    def detach(self,observer : IObserver) -> None :
        logging.info(f'OBSERVER PATTERN : {observer.name} : STOPS OBSERVING {self.name}')
        self._observers.remove(observer)
        
    def notify(self,info) -> None :
        logging.info(f'OBSERVER PATTERN : {self.name} : {len(self._observers)} OBSERVERS')
        
        for observer in self._observers:
            logging.info(f'OBSERVER PATTERN : {self.name} : NOTIFIES {observer.name}')
            observer.react(self.name,info)
            
    ####################################
    # button state
    ####################################
    def _button_inactive(self,button : Button) -> None :
        button.style.button_color=Color(None).name
        button.disabled = True
        
    def _button_active(self,button : Button,color : str = 'pink') -> None :
        button.style.button_color=color
        button.disabled = False
        
    def _update_button_state(self) -> None :
        if self.applied_settings == self._current_settings:
            # if nothing changed
            self._button_inactive(self._control_widgets[0])
            self._button_inactive(self._control_widgets[1])
        else:
            # if something changed
            self._button_active(self._control_widgets[0],color = 'lightgreen')
            self._button_active(self._control_widgets[1],color = 'pink')

    ####################################
    # button click
    ####################################
    def _click_apply(self,change) -> None :
        logging.debug(f'{self.name} : CLICKED {change.description}')
        self._deltas.clear()
        self._update_reference()                       # apply changes
        self.notify(self._deltas)                      # notify observers of changes
        
    '''reset to last applied values'''
    def _click_cancel(self,change) -> None :
        logging.debug(f'{self.name} : CLICKED {change.description}')
        for widget,value in zip(self._option_widgets,self.applied_settings):
            widget.value = value

    def _click_reset(self,change) -> None :
        logging.debug(f'{self.name} : CLICKED {change.description}')
        for i,(option_name,resettable) in enumerate(SETTINGS._option_names.items()):
            if resettable:
                self._option_widgets[i].value = self._default_settings[i]

    def _click_rescan(self,change) -> None :
        self._click_cancel(change)                     # forget changes
        self._deltas.clear()                           # remember changes
        self._update_backtests()                       # rescan directory
        self._logging(self._deltas,'DELTAS')
        self._logging(self.reference,'REFERENCE')
        self.notify(self._deltas)                      # notify observers of changes

    def _click_reload(self,change) -> None :
        self._click_cancel(change)                     # forget changes
        self.notify({'reload_data':True})              # message to trash data in self._DATA_CONTAINER.data_dico + reload
     
    def _click_kill(self,change) -> None :
        PID = os.getpid()
        if hasattr(signal, 'CTRL_C_EVENT'):
            # windows
            os.kill(PID, signal.SIGTERM)
        else:
            # unix
            PGID = os.getpgid(PID)
            os.killpg(PGID, signal.SIGKILL)

    ####################################
    # widget onchange
    ####################################
    def _change_title(self,title):
        return r'Javascript(''document.title="'+title+'"'')'
        
    def _onchange_app_name(self,change): 
        logging.debug(f'{self.name} : CHANGED {change.owner.description} = {change.new}')
        self._change_title(change.new)                          # TODO doesn't work, Javascript() calls wrapped within a function don't work, when used directly in a cell however it's fine, strange...
        self._deltas.clear()
        self._update_reference(filtered_options = ['app_name']) # apply changes
        self.notify(self._deltas)                               # notify observers of changes
     
    def _onchange_button_colour(self,change) -> None :
        logging.debug(f'{self.name} : CHANGED {change.owner.description} = {change.new}')
        self._update_button_state() # update button state
        
        
    ####################################
    # build GUI
    ####################################
    def _make_widget(self) -> VBox :
        # controls
        controls = HBox(self._make_controls())
        
        # options
        options = VBox(
            children    = self._make_options(),
            layout      = Layout(height='300px')
        )
        
        # return
        return VBox(
            children    = [controls,options],
            layout      = Layout(border='1px solid black',width=self._width,overflow_x='scroll')
        )
    
    def _make_controls(self) -> BUTTONS :
        button_apply = Button(
            description = 'apply',
            disabled    = True,
            tooltip     = 'apply changes',
        )
        button_apply.on_click(self._click_apply)

        button_cancel = Button(
            description = 'cancel',
            disabled    = True,
            tooltip     = 'retrieve last applied values',
        )
        button_cancel.on_click(self._click_cancel)

        button_reset = Button(
            description = 'reset',
            tooltip     = 'reset option values (excluding `app name` + path` + `view type`)',
        )
        button_reset.on_click(self._click_reset)

        button_rescan = Button(
            description = 'rescan',
            tooltip     = 'rescan backtest path',
        )
        button_rescan.on_click(self._click_rescan)

        button_reload = Button(
            description = 'reload',
            tooltip     = 'reload data from selected backtests',
        )
        button_reload.on_click(self._click_reload)

        button_kill = Button(
            description = 'kill',
            tooltip     = 'kill process',
        )
        button_kill.on_click(self._click_kill)

        return [
            button_apply,
            button_cancel,
            button_reset,
            button_rescan,
            button_reload,
            button_kill,
        ]

    def _make_options(self) -> WIDGETS:
        # app_name
        app_name = Text(
            description = 'app name',
            value       = SETTINGS._default_app_name,
        )
        app_name.observe(self._onchange_app_name,names='value')
        
        # backtest_path
        backtest_path = Text(
            description = 'path',
            value       = SETTINGS._default_backtest_path,
            disabled    = False,
            #continuous_update=False,
        )
        backtest_path.observe(self._onchange_button_colour,names='value')
        
        # view_type
        view_type = Dropdown(description='view type',options=SETTINGS._view_types)
        view_type.observe(self._onchange_button_colour,names='value')
        
        # plot_method
        plot_method = Dropdown(description='plot method',options=SINGLE_PLOT._plot_methods)
        plot_method.observe(self._onchange_button_colour,names='value')
        
        # return_type
        return_type = Dropdown(description='return type',options=SETTINGS._return_types)
        return_type.observe(self._onchange_button_colour,names='value')
        
        # date_from
        date_from = DatePicker(description='date from')
        date_from.observe(self._onchange_button_colour,names='value')
        
        # date_to
        date_to = DatePicker(description='date to')
        date_to.observe(self._onchange_button_colour,names='value')

        # time_from
        time_from = TimePicker(value=time(8,0),description='time from',step=1)
        time_from.observe(self._onchange_button_colour,names='value')
        
        # time_to
        time_to = TimePicker(value=time(17,0),description='time to',step=1)
        time_to.observe(self._onchange_button_colour,names='value')
            
        # instantiate and return list of widgets / options
        return list(map(eval,SETTINGS._option_names.keys()))

    ####################################
    # properties
    ####################################
    @property
    def _control_widgets(self) -> BUTTONS :
        return self.widget.children[0].children
    
    @property
    def _option_widgets(self) -> WIDGETS :
        return self.widget.children[1].children
    
    @property
    def _current_settings(self) -> List :
        return [x.value for x in self._option_widgets]
    
    ####################################
    # functions
    ####################################
    '''
    # TODO assumes settings are correct, backtest_path can be wrong!
    '''
    def _update_reference(self,
        force_populate     : bool      = False,
        filtered_options   : List[str] = None,
        ) -> Dict :
        logging.debug(f'{self.name} : _update_reference {force_populate} {filtered_options}')
        
        # find changes
        for i,(option_name,current_value) in enumerate(zip(SETTINGS._option_names,self._current_settings)):
            # apply filter as needed
            should_consider = True if filtered_options == None else (option_name in filtered_options)
            
            # update as required
            if should_consider:
                logging.debug(f'{self.name} : _update_reference {option_name}')
                if option_name == 'backtest_path':
                    if force_populate | (Path(self.applied_settings[i]) != Path(current_value)):
                        self.applied_settings[i]        = current_value       # update checkpoint
                        self.reference[option_name]     = current_value       # update reference
                        self._deltas[option_name]       = current_value       # log change

                        # update all_backtests and all_books
                        self._update_backtests()
                else:
                    if force_populate | (self.applied_settings[i] != current_value):
                        self.applied_settings[i]        = current_value       # update checkpoint
                        self.reference[option_name]     = current_value       # update reference
                        self._deltas[option_name]       = current_value       # log change

        # update button state
        self._update_button_state()

        # forget changes
        if force_populate:
            self._deltas.clear()
            
        # logging
        self._logging(self._deltas,'DELTAS')
        self._logging(self.reference,'REFERENCE')
        
    def _update_backtests(self) -> None:
        logging.info(f'{self.name} : _update_reference all_backtests')
        self.reference['all_backtests']             = self._rescan_all_backtests()
        self._deltas['all_backtests']               = self.reference['all_backtests']

        logging.info(f'{self.name} : _update_reference all_books')
        self.reference['all_books']                 = self._rescan_all_books() # TODO shouldnt be here!
        self._deltas['all_books']                   = self.reference['all_books']
        
    def _rescan_all_backtests(self) -> ID_PATHS :
        p = self.reference['backtest_path']
        return {x.stem:x for x in Path(p).iterdir()}
        
    def _rescan_all_books(self) -> ID_PATHS :
        # TODO update this
        return {x:x for x in ['Trading','Quote','MM2','Hedge','Hit']}

    def _logging(self,dico,name) -> None:
        for k,v in dico.items():
            logging.debug(f'{self.name} : {name} : {k} : {v}')

In [ ]:
# __file__ exists if notebook called with %run
# e.g. only do example if called directly
try:
    __file__
except NameError:
    # example
    a=SETTINGS()
    display(dir(a))
    display(a.widget)

In [ ]:
'''
a._change_title('x')

Javascript('document.title="{}"'.format('ab'))

Javascript(r'document.title="'+'a'+'"')

Javascript(r'document.title='+'a'+'')

def rename(title):
    Javascript('document.title="'+title+'"')
'''
None

In [ ]:
# write to console
'''
import os
os.write(1, "text\n".encode())
'''
None